# 1) Define the necessary imports for my code

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

import torchvision.transforms as transforms

import numpy as np

import matplotlib.pyplot as plt

# 2) Copy the NumpyImageDataset.py code into this notebook

In [12]:
from __future__ import print_function
from PIL import Image

class NumpyImageDataset(data.Dataset):

    # image dataset from numpy arrays

    # Example:

    # num_classes = 20
    # train_data = np.load('../data/cifar_train_data.npy').transpose((0,2,3,1))
    # train_label = np.load('../data/cifar_train_label.npy')
    # trainset = NumpyImageDataset(train_data, train_label, transform=transform_train)
    # trainloader = data.DataLoader(trainset, batch_size=args.train_batch, shuffle=True, num_workers=args.workers)    

    def __init__(self, data_array, label_array, transform=None, label_transform=None):
        self.data_tensor = data_array
        self.label_tensor = torch.from_numpy(label_array).type('torch.LongTensor')
        self.transform = transform
        self.label_transform = label_transform
        assert self.data_tensor.shape[0] == self.label_tensor.size(0)

    def __getitem__(self, index):

        img, label = self.data_tensor[index], self.label_tensor[index]

        # change to PIL image
        img = Image.fromarray(img)

        if self.transform is not None:
            # apply user-defined sequence of transformations
            img = self.transform(img)

        if self.label_transform is not None:
            label = self.label_transform(label)

        return img, label

    def __len__(self):
        return self.data_tensor.shape[0]

# 3) Copy the generate_submission.py code into this notebook

In [13]:
import sys

# You can also import his python and call the following function
# to generate the submission file
def writeSubmissionFile(labels, fn):
    f = open(fn, 'w')
    f.write("id,category\n")
    for ii, ll in enumerate(labels):
        f.write(str(ii) + "," + str(int(ll)) + "\n")

    f.close()

# 4) Define the transforms, and create the 'train_loader', 'validation_loader', and 'test_loader' variables for later

In [14]:
# Define the transforms for our training dataset
transform_train = transforms.Compose([transforms.ToTensor(),
                                      transforms.Normalize(mean = 0.5, std = 1),
                                      transforms.RandomGrayscale(p=0.2),
                                      transforms.RandomHorizontalFlip(p=0.2),
                                      transforms.RandomApply([transforms.GaussianBlur(kernel_size=3)],p=0.2),
                                      transforms.RandomCrop(32, padding=4),
                                      transforms.RandomRotation(10)
                                     ])

# Define the transforms for our validation and test datasets
transform_test = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize(mean = 0.5, std = 1)])

# Determine the size of the training set
train_size = 45000

# Import the training data and label and split it into a train set and validation set
initial_data = np.load('/kaggle/input/model-data/cifar_train_data.npy').transpose((0,2,3,1))
initial_target = np.load('/kaggle/input/model-data/cifar_train_label.npy')

train_data, train_target = initial_data[:train_size], initial_target[:train_size]
validation_data, validation_target = initial_data[train_size:], initial_target[train_size:]

train_set = NumpyImageDataset(train_data, train_target, transform = transform_train)
train_loader = data.DataLoader(train_set, batch_size = 32, shuffle = True)

validation_set = NumpyImageDataset(validation_data, validation_target, transform = transform_test)
validation_loader = data.DataLoader(validation_set, batch_size = 32)

# Import the testing data
test_data = np.load('/kaggle/input/model-data/cifar_test_data.npy').transpose((0,2,3,1))
test_target = np.zeros(10000)
test_set = NumpyImageDataset(test_data, test_target, transform = transform_test)
test_loader = data.DataLoader(test_set, batch_size = 1)

# 5) Define some useful functions for training our model

In [15]:
# Carry out one training step of the model
def train_step(model, batch):
    images, targets = batch 
    outputs = model(images)                  
    loss = F.cross_entropy(outputs, targets) 
    return loss

# Carry out one set of predictions on the validation set
def validation_step(model, batch):
    images, targets = batch
    outputs = model(images)
    acc = accuracy(outputs, targets)
    return acc

# Determine the accuracy of predictions
def accuracy(outputs, targets):
    predictions = torch.max(outputs, dim=1)[1]
    return torch.tensor(torch.sum(predictions == targets).item() / len(predictions))

<h1> 6) Define the architecture of my Residual Network </h1>
<h3> Details: 10 Layers: 9 convolutional and 1 linear; 2 residual blocks </h3>

In [16]:
class ResNet(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Layer 1
        self.conv1 = nn.Sequential(nn.Conv2d(3, 32, kernel_size = 3, padding = 1), 
                                   nn.BatchNorm2d(32), 
                                   nn.ReLU(inplace = True))
        
        # Layer 2
        self.conv2 = nn.Sequential(nn.Conv2d(32, 64, kernel_size = 3, padding = 1), 
                                   nn.BatchNorm2d(64), 
                                   nn.ReLU(inplace = True))
        
        # Layer 3
        self.conv3 = nn.Sequential(nn.Conv2d(64, 128, kernel_size = 3, padding = 1), 
                                   nn.BatchNorm2d(128), 
                                   nn.ReLU(inplace = True),
                                   nn.MaxPool2d(2))
        
        # Layer 4 and 5
        self.resBlock1 = nn.Sequential(nn.Conv2d(128, 128, kernel_size = 3, padding = 1), 
                                       nn.BatchNorm2d(128), 
                                       nn.ReLU(inplace = True), 
                                       nn.Conv2d(128, 128, kernel_size = 3, padding = 1), 
                                       nn.BatchNorm2d(128), 
                                       nn.ReLU(inplace = True))
        
        # Layer 6
        self.conv4 = nn.Sequential(nn.Conv2d(128, 256, kernel_size = 3, padding = 1), 
                                   nn.BatchNorm2d(256), 
                                   nn.ReLU(inplace = True),
                                   nn.MaxPool2d(2))
        
        # Layer 7
        self.conv5 = nn.Sequential(nn.Conv2d(256, 512, kernel_size = 3, padding = 1), 
                                   nn.BatchNorm2d(512), 
                                   nn.ReLU(inplace = True),
                                   nn.MaxPool2d(2))
        
        # Layer 8 and 9
        self.resBlock2 = nn.Sequential(nn.Conv2d(512, 512, kernel_size = 3, padding = 1), 
                                       nn.BatchNorm2d(512), 
                                       nn.ReLU(inplace = True), 
                                       nn.Conv2d(512, 512, kernel_size = 3, padding = 1), 
                                       nn.BatchNorm2d(512), 
                                       nn.ReLU(inplace = True))
        
        # Layer 10
        self.linear1 = nn.Sequential(nn.MaxPool2d(4), 
                                     nn.Flatten(), 
                                     nn.Linear(512, 20))
        
    def forward(self, xb):
        out = self.conv1(xb)
        out = self.conv2(out)
        out = self.conv3(out)
        out = self.resBlock1(out) + out
        out = self.conv4(out)
        out = self.conv5(out)
        out = self.resBlock2(out) + out
        out = self.linear1(out)
        return out

# 7) Define the necessary methods to load data so that it can be processed with a GPU

In [17]:
# Define the to_device method
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

# Define the GPUDataLoader class
class GPUDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

# Define the GPU we want to use
gpu = torch.device('cuda')

# Load the data with the GPUDataLoader class
train_loader = GPUDataLoader(train_loader, gpu)
validation_loader = GPUDataLoader(validation_loader, gpu)
test_loader = GPUDataLoader(test_loader, gpu)

# 8) Define the 'fit' method for training our dataset

In [18]:
# Define the 'fit' function which trains our model on the training and validation set
def fit(epochs, lr, model, train_loader, validation_loader, opt_func = torch.optim.Adam):
    optimizer = opt_func(model.parameters(), lr)
    
    # Train the model for the given number of epochs
    for epoch in range(epochs):
        
        # Training the model 
        model.train()
        train_losses = []
        for batch in train_loader:
            loss = train_step(model, batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        
        # Test the model on the validation dataset
        model.eval()
        outputs = [validation_step(model, batch) for batch in validation_loader]
        result = np.mean(outputs)
        print("Epoch:", epoch + 1, "Validation Accuracy: {:.2f}".format(100 * result))

    print("The model has been trained")

# 9) Train our model and define the hyperparameters

In [19]:
# Instantiate the model
model = to_device(ResNet(), gpu)

# Define some hyperparameters
num_epochs = 50
opt_func = torch.optim.Adam
lr = 0.0001

# Train the model
print("Batch Size:", 32, "Learning Rate:", lr)
fit(num_epochs, lr, model, train_loader, validation_loader, opt_func)

Batch Size: 32 Learning Rate: 0.0001
Epoch: 1 Validation Accuracy: 47.61
Epoch: 2 Validation Accuracy: 57.42
Epoch: 3 Validation Accuracy: 61.05
Epoch: 4 Validation Accuracy: 65.09
Epoch: 5 Validation Accuracy: 68.19
Epoch: 6 Validation Accuracy: 69.51
Epoch: 7 Validation Accuracy: 70.26
Epoch: 8 Validation Accuracy: 72.03
Epoch: 9 Validation Accuracy: 72.89
Epoch: 10 Validation Accuracy: 74.88
Epoch: 11 Validation Accuracy: 74.26
Epoch: 12 Validation Accuracy: 76.61
Epoch: 13 Validation Accuracy: 77.69
Epoch: 14 Validation Accuracy: 76.71
Epoch: 15 Validation Accuracy: 77.67
Epoch: 16 Validation Accuracy: 77.65
Epoch: 17 Validation Accuracy: 78.74
Epoch: 18 Validation Accuracy: 79.48
Epoch: 19 Validation Accuracy: 79.58
Epoch: 20 Validation Accuracy: 79.20
Epoch: 21 Validation Accuracy: 78.78
Epoch: 22 Validation Accuracy: 79.86
Epoch: 23 Validation Accuracy: 80.45
Epoch: 24 Validation Accuracy: 81.43
Epoch: 25 Validation Accuracy: 80.33
Epoch: 26 Validation Accuracy: 81.85
Epoch: 27 

# 10) Use our trained model to make predictions on the test set, and write the predictions to a .csv file

In [20]:
ids = np.arange(0, len(test_set))
predictions = np.array([0])

for data, targets in test_loader:
    output = model(data)
    preds = torch.max(output, dim=1)[1]
    preds = preds.detach().cpu().numpy()
    predictions = np.concatenate((predictions, preds))

predictions = np.delete(predictions, 0)
print(predictions)

results = np.column_stack((ids, predictions))
print(results)

writeSubmissionFile(predictions, 'emilio_moya_submissions.csv')

[ 7 11 16 ...  0  2  8]
[[   0    7]
 [   1   11]
 [   2   16]
 ...
 [9997    0]
 [9998    2]
 [9999    8]]
